# 实验二：命名实体识别

**实验任务：**
1. 阅读、理解并运行本次实验提供的利用HMM和CRF进行命名实体识别的代码。 （1分）
1. 在本Notebook中指定位置，实现一个命名实体识别的评价程序，该程序可以在实体级别计算测试结果的每种实体以及总体的Precision、Recall和F1值。（3分）
1. 根据给定的提示，在本Notebook中指定位置，实现一个基于最大熵模型的实体识别系统，并利用ner_char_data目录下的train.txt文件训练模型，利用test.txt文件测试模型效果。（5分）
1. 在本Notebook中指定位置，利用给定的ner_clue_data目录下的训练数据文件train.txt分别训练HMM、ME和CRF模型,并使用dev.txt文件里的数据来测试三个模型。输出每个模型在dev.txt数据上的测试结果（每个模型对应一个结果文件），在Notebook中输出每个模型对应的每种实体以及总体的Precision、Recall和F1值。（6分）
1. 按照课程QQ群里给定的实验指导书，学习利用华为云上的计算资源进行命名实体识别（附加分：2分）
1. 按照课程QQ群里给定的模板完成本次实验的实验报告，并按照要求提交。 （5分）

**实验提交截止时间：**
* 2021年12月29日 晚上10点

**实验提交方式：**
* 在百度AI Studio的课程中提交，实验报告作为附件上传。

#  1. 数据读取
## 1.1对 ner_char_data目录下的数据进行读取
ner_char_data 目录下数据为预处理后的数据，包括训练数据文件train.txt和测试数据文件test.txt。数据文件中的每一行由两部分组成字符和字符对应的实体标记，两部分之间用空格分隔。实体标记采用BMES的方式。文件中不同的句子之间用空行分割。

In [1]:
""" 读取并处理 ner_char_data 目录下的数据文件 """
def data_build(file_name: str, make_vocab=True):
    word_lists = []
    tag_lists = []
    with open(file_name, "r", encoding="utf-8") as file_read:
        word_list = []
        tag_list = []
        for line in file_read:
            if line != "\n":
                word, tag = line.strip("\n").split()
                word_list.append(word)
                tag_list.append(tag)
            else:
                word_lists.append(word_list)
                tag_lists.append(tag_list)
                word_list = []
                tag_list = []

    if make_vocab:
        word2id = {}
        for word_list in word_lists:
            for word in word_list:
                if word not in word2id:
                    word2id[word] = len(word2id)
        tag2id = {}
        for tag_list in tag_lists:
            for tag in tag_list:
                if tag not in tag2id:
                    tag2id[tag] = len(tag2id)
        return word_lists, tag_lists, word2id, tag2id
    return word_lists, tag_lists


train_word_lists, train_tag_lists, word2id, tag2id = data_build(file_name="ner_char_data/train.txt", make_vocab=True)
test_word_lists, test_tag_lists = data_build(file_name="ner_char_data/test.txt", make_vocab=False)

## 2.1 ner_clue_data目录下的文件中的数据格式
  ### 数据类别：
    数据分为10个标签类别，分别为: 地址（address），书名（book），公司（company），游戏（game），政府（government），电影（movie），姓名（name），组织机构（organization），职位（position），景点（scene）

  ### 标签类别定义 & 标注规则：
    地址（address）: **省**市**区**街**号，**路，**街道，**村等（如单独出现也标记）。地址是标记尽量完全的, 标记到最细。
    书名（book）: 小说，杂志，习题集，教科书，教辅，地图册，食谱，书店里能买到的一类书籍，包含电子书。
    公司（company）: **公司，**集团，**银行（央行，中国人民银行除外，二者属于政府机构）, 如：新东方，包含新华网/中国军网等。
    游戏（game）: 常见的游戏，注意有一些从小说，电视剧改编的游戏，要分析具体场景到底是不是游戏。
    政府（government）: 包括中央行政机关和地方行政机关两级。 中央行政机关有国务院、国务院组成部门（包括各部、委员会、中国人民银行和审计署）、国务院直属机构（如海关、税务、工商、环保总局等），军队等。
    电影（movie）: 电影，也包括拍的一些在电影院上映的纪录片，如果是根据书名改编成电影，要根据场景上下文着重区分下是电影名字还是书名。
    姓名（name）: 一般指人名，也包括小说里面的人物，宋江，武松，郭靖，小说里面的人物绰号：及时雨，花和尚，著名人物的别称，通过这个别称能对应到某个具体人物。
    组织机构（organization）: 篮球队，足球队，乐团，社团等，另外包含小说里面的帮派如：少林寺，丐帮，铁掌帮，武当，峨眉等。
    职位（position）: 古时候的职称：巡抚，知州，国师等。现代的总经理，记者，总裁，艺术家，收藏家等。
    景点（scene）: 常见旅游景点如：长沙公园，深圳动物园，海洋馆，植物园，黄河，长江等。
  
  ### 数据来源地址：
  <a href='https://www.cluebenchmarks.com/introduce.html'>数据下载</a>
    
  ### 数据分布：
    训练集(train.txt)：10748
    验证集集(dev.txt)：1343

    按照不同标签类别统计，训练集数据分布如下（注：一条数据中出现的所有实体都进行标注，如果一条数据出现两个地址（address）实体，那么统计地址（address）类别数据的时候，算两条数据）：
    【训练集】标签数据分布如下：
    地址（address）:2829
    书名（book）:1131
    公司（company）:2897
    游戏（game）:2325
    政府（government）:1797
    电影（movie）:1109
    姓名（name）:3661
    组织机构（organization）:3075
    职位（position）:3052
    景点（scene）:1462

    【验证集】标签数据分布如下：
    地址（address）:364
    书名（book）:152
    公司（company）:366
    游戏（game）:287
    政府（government）:244
    电影（movie）:150
    姓名（name）:451
    组织机构（organization）:344
    职位（position）:425
    景点（scene）:199


  ## 数据字段解释：
    以train.json为例，数据分为两列：text & label，其中text列代表文本，label列代表文本中出现的所有包含在10个类别中的实体。
    例如：
      text: "北京勘察设计协会副会长兼秘书长周荫如"
      label: {"organization": {"北京勘察设计协会": [[0, 7]]}, "name": {"周荫如": [[15, 17]]}, "position": {"副会长": [[8, 10]], "秘书长": [[12, 14]]}}
      其中，organization，name，position代表实体类别，
      "organization": {"北京勘察设计协会": [[0, 7]]}：表示原text中，"北京勘察设计协会" 是类别为 "组织机构（organization）" 的实体, 并且start_index为0，end_index为7 （注：下标从0开始计数）
      "name": {"周荫如": [[15, 17]]}：表示原text中，"周荫如" 是类别为 "姓名（name）" 的实体, 并且start_index为15，end_index为17
      "position": {"副会长": [[8, 10]], "秘书长": [[12, 14]]}：表示原text中，"副会长" 是类别为 "职位（position）" 的实体, 并且start_index为8，end_index为10，同时，"秘书长" 也是类别为 "职位（position）" 的实体,
      并且start_index为12，end_index为14

## 数据来源：
    本数据是在清华大学开源的文本分类数据集THUCTC基础上，选出部分数据进行细粒度命名实体标注，原数据来源于Sina News RSS.

  

In [4]:
""" 在这里练习读取并处理 ner_clue_data目录下的数据。"""

import json

clue_files = ["ner_clue_data/train.txt", "ner_clue_data/dev.txt"]
char_files = ["ner_clue_data/train_char.txt", "ner_clue_data/dev_char.txt"]
for j in range(0, len(clue_files)):
    with open(char_files[j], "w") as wf:
        with open(clue_files[j], "r") as f:
            for line_str in f:
                line: dict = json.loads(line_str)
                text: str = line["text"]
                tag_list: list = ["O"] * len(text)
                label: dict = line["label"]
                for label_item in label.keys():
                    for item in label[label_item]:
                        for lst in label[label_item][item]:
                            for i in range(lst[0], lst[1] + 1):
                                if i == lst[0]:
                                    tag_list[i] = "B-" + label_item
                                elif i == lst[1]:
                                    tag_list[i] = "E-" + label_item
                                else:
                                    tag_list[i] = "M-" + label_item

                for i in range(0, len(text)):
                    wf.write(text[i] + " " + tag_list[i] + "\n")
                wf.write("\n")


# 2. 隐马尔科夫（HMM）模型
## 2.1隐马尔科夫模型参数的计算
利用训练数据获取HMM模型对应的参数**A**，**B**和**Pi**

In [5]:
""" HMM 参数构建 """
import numpy as np
# N: 状态数，这里对应存在的标注的种类 
# M: 观测数，这里对应有多少不同的字
N, M = len(tag2id), len(word2id)
# 状态转移概率矩阵 A[i][j]表示从i状态转移到j状态的概率
A = np.zeros(shape=(N, N), dtype=float)
# 观测概率矩阵, B[i][j]表示i状态下生成j观测的概率
B = np.zeros(shape=(N, M), dtype=float)
# 初始状态概率  Pi[i]表示初始时刻为状态i的概率
Pi = np.zeros(shape=N, dtype=float)

""" 构建转移概率矩阵 """
for tag_list in train_tag_lists:
    seq_len = len(tag_list)
    for i in range(seq_len - 1):
        current_tagid = tag2id[tag_list[i]]
        next_tagid = tag2id[tag_list[i+1]]
        A[current_tagid][next_tagid] += 1
A[A == 0.] = 1e-10  # 平滑处理
A = A / np.sum(a=A, axis=1, keepdims=True)

""" 构建观测概率矩阵 """
for tag_list, word_list in zip(train_tag_lists, train_word_lists):
    assert len(tag_list) == len(word_list)
    for tag, word in zip(tag_list, word_list):
        tag_id = tag2id[tag]
        word_id = word2id[word]
        B[tag_id][word_id] += 1
B[B == 0.] = 1e-10  # 平滑处理
B = B / np.sum(a=B, axis=1, keepdims=True)

""" 构建初始状态概率 """
for tag_list in train_tag_lists:
    init_tagid = tag2id[tag_list[0]]
    Pi[init_tagid] += 1
Pi[Pi == 0.] = 1e-10  # 平滑处理
Pi = Pi / np.sum(a=Pi)


## 2.2 维特比算法的实现

In [6]:
""" 维特比算法 """
def viterbi(word_list, word2id, tag2id):
    """
    使用维特比算法对给定观测序列求状态序列， 这里就是对字组成的序列,求其对应的标注。
    维特比算法实际是用动态规划解隐马尔可夫模型预测问题，即用动态规划求概率最大路径（最优路径）
    这时一条路径对应着一个状态序列
    """
    # 问题:整条链很长的情况下，十分多的小概率相乘，最后可能造成下溢
    # 解决办法：采用对数概率，这样源空间中的很小概率，就被映射到对数空间的大的负数
    #  同时相乘操作也变成简单的相加操作
    ALog = np.log(A)
    BLog = np.log(B)
    PiLog = np.log(Pi)

    # 初始化 维比特矩阵viterbi 它的维度为[状态数, 序列长度]
    # 其中viterbi[i, j]表示标注序列的第j个标注为i的所有单个序列(i_1, i_2, ..i_j)出现的概率最大值
    seq_len = len(word_list)
    viterbi = np.zeros(shape=(N, seq_len), dtype=float)
    # backpointer是跟viterbi一样大小的矩阵
    # backpointer[i, j]存储的是 标注序列的第j个标注为i时，第j-1个标注的id
    # 等解码的时候，我们用backpointer进行回溯，以求出最优路径
    backpointer = np.zeros(shape=(N, seq_len), dtype=float)

    # Pi[i] 表示第一个字的标记为i的概率
    # Bt[word_id]表示字为word_id的时候，对应各个标记的概率
    # A.t()[tag_id]表示各个状态转移到tag_id对应的概率

    # 所以第一步为
    start_wordid = word2id.get(word_list[0], None)
    Bt = BLog.T
    if start_wordid is None:
        # 如果字不再字典里，则假设状态的概率分布是均匀的
        bt = np.log(np.ones(shape=N, dtype=float) / N)
    else:
        bt = Bt[start_wordid]
    viterbi[:, 0] = PiLog + bt
    backpointer[:, 0] = -1

    # 递推公式：viterbi[tag_id, step] = max(viterbi[:, step-1]* A.t()[tag_id] * Bt[word])
    # 其中word是step时刻对应的字, 由上述递推公式求后续各步
    for step in range(1, seq_len):
        wordid = word2id.get(word_list[step], None)
        # 处理字不在字典中的情况
        # bt是在t时刻字为wordid时，状态的概率分布
        if wordid is None:
            # 如果字不再字典里，则假设状态的概率分布是均匀的
            bt = np.log(np.ones(N) / N)
        else:
            bt = Bt[wordid]  # 否则从观测概率矩阵中取bt
        for tag_id in range(len(tag2id)):
            max_prob = np.max(a=viterbi[:, step - 1] + ALog[:, tag_id], axis=0)
            max_id = np.argmax(a=viterbi[:, step - 1] + ALog[:, tag_id], axis=0)
            viterbi[tag_id, step] = max_prob + bt[tag_id]
            backpointer[tag_id, step] = max_id

    # 终止， t=seq_len 即 viterbi[:, seq_len]中的最大概率，就是最优路径的概率
    best_path_prob = np.max(a=viterbi[:, seq_len - 1], axis=0)
    best_path_pointer = np.argmax(a=viterbi[:, seq_len - 1], axis=0)

    # 回溯，求最优路径
    best_path_pointer = int(best_path_pointer)
    best_path = [best_path_pointer]

    for back_step in range(seq_len-1, 0, -1):
        best_path_pointer = backpointer[best_path_pointer, back_step]
        best_path_pointer = int(best_path_pointer)
        best_path.append(best_path_pointer)

    # 将tag_id组成的序列转化为tag
    assert len(best_path) == len(word_list)
    id2tag = dict((id_, tag) for tag, id_ in tag2id.items())
    tag_list = [id2tag[id_] for id_ in reversed(best_path)]

    return tag_list

## 2.3 利用HMM模型和viterbi进行实体识别

In [7]:
""" 利用HMM识别ner_char_data目录下test.txt中的数据"""
pred_tag_lists = []
for word_list in test_word_lists:
    pred_tag_list = viterbi(word_list, word2id, tag2id)
    pred_tag_lists.append(pred_tag_list)

## 2.4 按标记对HMM的识别结果进行评测

In [8]:
""" HMM 评测 """
from evaluating import Metrics
metrics = Metrics(test_tag_lists, pred_tag_lists, remove_O=False)
metrics.report_scores()
metrics.report_confusion_matrix()

           precision    recall  f1-score   support
   B-NAME     0.9800    0.8750    0.9245       112
   M-NAME     0.9459    0.8537    0.8974        82
   E-NAME     0.9000    0.8036    0.8491       112
        O     0.9568    0.9177    0.9369      5190
    B-PRO     0.5581    0.7273    0.6316        33
    E-PRO     0.6512    0.8485    0.7368        33
    B-EDU     0.9000    0.9643    0.9310       112
    E-EDU     0.9167    0.9821    0.9483       112
  B-TITLE     0.8811    0.8925    0.8867       772
  M-TITLE     0.9038    0.8751    0.8892      1922
  E-TITLE     0.9514    0.9637    0.9575       772
    B-ORG     0.8422    0.8879    0.8644       553
    M-ORG     0.9002    0.9327    0.9162      4325
    E-ORG     0.8262    0.8680    0.8466       553
   B-CONT     0.9655    1.0000    0.9825        28
   M-CONT     0.9815    1.0000    0.9907        53
   E-CONT     0.9655    1.0000    0.9825        28
    M-EDU     0.9348    0.9609    0.9477       179
   B-RACE     1.0000    0.9286 

# 3. 条件随机场(CRF)模型

## 3.1 安装sklearn-crfsuite

In [9]:
!pip install sklearn-crfsuite

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 747kB 1.8MB/s eta 0:00:01


## 3.2 从sklearn_crfsuite模块中导入CRF包

In [10]:
from sklearn_crfsuite import CRF

## 3.3  CRF中的特征抽取

In [11]:
def word2features(sent, i):
    """抽取单个字的特征"""
    word = sent[i]
    prev_word = "<s>" if i == 0 else sent[i-1]
    next_word = "</s>" if i == (len(sent)-1) else sent[i+1]
    # 使用的特征：
    # 前一个词，当前词，后一个词，
    # 前一个词+当前词， 当前词+后一个词
    features = {
        'w': word,
        'w-1': prev_word,
        'w+1': next_word,
        'w-1:w': prev_word+word,
        'w:w+1': word+next_word,
        'w-1:w:w+1':prev_word+word+next_word,
        'bias': 1
    }
    return features


def sent2features(sent):
    """抽取序列特征"""
    return [word2features(sent, i) for i in range(len(sent))]

## 3.4  CRF模型的实现

In [12]:
class CRFModel(object):
    def __init__(self,
                 algorithm='lbfgs',
                 c1=0.1,
                 c2=0.1,
                 max_iterations=100,
                 all_possible_transitions=False
                 ):

        self.model = CRF(algorithm=algorithm,
                         c1=c1,
                         c2=c2,
                         max_iterations=max_iterations,
                         all_possible_transitions=all_possible_transitions)

    def train(self, sentences, tag_lists):
        features = [sent2features(s) for s in sentences]
        self.model.fit(features, tag_lists)

    def test(self, sentences):
        features = [sent2features(s) for s in sentences]
        pred_tag_lists = self.model.predict(features)
        return pred_tag_lists

## 3.3  CRF模型训练、测试与评价

In [13]:
from evaluating import Metrics
# 训练CRF模型
crf_model = CRFModel()
crf_model.train(train_word_lists, train_tag_lists)

pred_tag_lists = crf_model.test(test_word_lists)

metrics = Metrics(test_tag_lists, pred_tag_lists, remove_O=False)
metrics.report_scores()
metrics.report_confusion_matrix()

           precision    recall  f1-score   support
   B-NAME     1.0000    0.9821    0.9910       112
   M-NAME     1.0000    0.9756    0.9877        82
   E-NAME     1.0000    0.9821    0.9910       112
        O     0.9653    0.9659    0.9656      5190
    B-PRO     0.9375    0.9091    0.9231        33
    E-PRO     0.9375    0.9091    0.9231        33
    B-EDU     0.9820    0.9732    0.9776       112
    E-EDU     0.9910    0.9821    0.9865       112
  B-TITLE     0.9417    0.9417    0.9417       772
  M-TITLE     0.9447    0.9069    0.9254      1922
  E-TITLE     0.9819    0.9819    0.9819       772
    B-ORG     0.9550    0.9584    0.9567       553
    M-ORG     0.9436    0.9630    0.9532      4325
    E-ORG     0.9189    0.9222    0.9206       553
   B-CONT     1.0000    1.0000    1.0000        28
   M-CONT     1.0000    1.0000    1.0000        53
   E-CONT     1.0000    1.0000    1.0000        28
    M-EDU     0.9824    0.9330    0.9570       179
   B-RACE     1.0000    1.0000 

# 4. 实现实体级别评价程序
请在下面的Cell中实现一个命名实体识别的评价程序，该程序可以在实体级别计算测试结果的每种实体以及总体的Precision、Recall和F1值。（3分）

In [14]:
# 请在这里实现实体级别评价程序

def calc_precision(result_file: str, answer_file: str) -> float:
    """计算识别分词结果准确率（识别结果中正确的比例）

    Args:
        result_file: 识别结果文件路径
        answer_file: 标准识别结果文件路径

    Returns:
        返回准确率
    """

    # 统计识别结果中有多少个实体
    result_ner_count: int = 0
    with open(result_file, "r") as f:
        for line in f:
            if "B-" in line:
                result_ner_count += 1

    # 对比结果和标准文件
    result_label_list: list[str]
    answer_label_list: list[str]
    with open(result_file, "r") as f:
        result_label_list = f.read().split("\n")
        for i in range(0, len(result_label_list)):
            result_label_list[i] = result_label_list[i][2:]
    with open(answer_file, "r") as af:
        answer_label_list = af.read().split("\n")
        for i in range(0, len(answer_label_list)):
            answer_label_list[i] = answer_label_list[i][2:]

    ner_start_line: list[int] = []
    ner_end_line: list[int] = []
    line_num: int = 0
    with open(answer_file, "r") as af:
        for aline in af:
            if "B-" in aline:
                ner_start_line.append(line_num)
            elif "E-" in aline:
                ner_end_line.append(line_num)
            line_num += 1
        if len(ner_start_line) != len(ner_end_line):
            print("===== Error =====")
            return -1

    ner_match_count: int = 0
    for i in range(0, len(ner_start_line)):
        start_line_num: int = ner_start_line[i]
        end_line_num: int = ner_end_line[i]
        matched: bool = True
        for j in range(start_line_num, end_line_num + 1):
            if result_label_list[j] != answer_label_list[j]:
                matched = False
                break
        if matched:
            ner_match_count += 1

    return ner_match_count / result_ner_count


def calc_recall(result_file: str, answer_file: str) -> float:
    """计算识别结果召回率（识别结果找出了正确实体的多少）

    Args:
        result_file: 识别结果文件路径
        answer_file: 标准识别结果文件路径

    Returns:
        返回召回率
    """

    # 统计正确实体有多少
    answer_ner_count: int = 0
    with open(answer_file, "r") as af:
        for aline in af:
            if "B-" in aline:
                answer_ner_count += 1

    # 对比结果和标准文件
    result_label_list: list[str]
    answer_label_list: list[str]
    with open(result_file, "r") as f:
        result_label_list = f.read().split("\n")
        for i in range(0, len(result_label_list)):
            result_label_list[i] = result_label_list[i][2:]
    with open(answer_file, "r") as af:
        answer_label_list = af.read().split("\n")
        for i in range(0, len(answer_label_list)):
            answer_label_list[i] = answer_label_list[i][2:]

    ner_start_line: list[int] = []
    ner_end_line: list[int] = []
    line_num: int = 0
    with open(answer_file, "r") as af:
        for aline in af:
            if "B-" in aline:
                ner_start_line.append(line_num)
            elif "E-" in aline:
                ner_end_line.append(line_num)
            line_num += 1
        if len(ner_start_line) != len(ner_end_line):
            print("===== Error =====")
            return -1

    ner_match_count: int = 0
    for i in range(0, len(ner_start_line)):
        start_line_num: int = ner_start_line[i]
        end_line_num: int = ner_end_line[i]
        matched: bool = True
        for j in range(start_line_num, end_line_num + 1):
            if result_label_list[j] != answer_label_list[j]:
                matched = False
                break
        if matched:
            ner_match_count += 1

    return ner_match_count / answer_ner_count


def calc_f(precision: float, recall: float) -> float:
    """计算准确率和召回率的调和平均
    Args:
        precision: 准确率
        recall:    召回率
    Returns:
        返回准确率和召回率的调和平均
    """

    return (2 * precision * recall) / (precision + recall)


def evaluate(result_file: str, answer_file: str):
    p: float = calc_precision(result_file, answer_file)
    r: float = calc_recall(result_file, answer_file)
    f: float = calc_f(p, r)
    print("presision = {}, recall = {}, f = {}".format(p, r, f))


# 5. 基于最大熵模型的实体识别
请在下面的Cell中实现一个基于最大熵模型的实体识别系统，并利用ner_char_data目录下的train.txt文件训练模型，利用test.txt文件测试模型效果。

In [15]:
# 请在这里实现一个基于最大熵模型的实体识别系统
# 导入sklearn中的LogisticRegression （LogisticRegression即为最大熵模型）
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from evaluating import Metrics


class MEModel:
    def __init__(self):
        self.model = LogisticRegression(penalty="l2", C=1.0)

    def train(self, train_word_lists, train_tag_lists, test_word_lists, test_tag_lists, test_word2id):
        all_train_words = []
        for t in train_word_lists:
            for tt in t:
                all_train_words.append(tt)

        test_word_lists_copy = []
        for i in range(0, len(test_word_lists)):
            r = []
            for j in range(0, len(test_word_lists[i])):
                if test_word_lists[i][j] in all_train_words:
                    r.append(test_word_lists[i][j])
                else:
                    r.append(test_word_lists[i][j] + str(test_word2id[test_word_lists[i][j]]))
            test_word_lists_copy.append(r)
        test_tag_lists_copy = []
        for i in range(0, len(test_tag_lists)):
            r = ["O"] * len(test_tag_lists[i])
            test_tag_lists_copy.append(r)

        word_lists = train_word_lists + test_word_lists_copy
        features = [sent2features(s) for s in word_lists]

        all_features = []
        for f in features:
            for ff in f:
                all_features.append(ff)
        v = DictVectorizer(sparse=True)
        X = v.fit_transform(all_features)

        all_tags = []
        tag_lists = train_tag_lists + test_tag_lists
        for t in tag_lists:
            for tt in t:
                all_tags.append(tt)
        self.model.fit(X, all_tags)

    def test(self, train_word_lists, test_word_lists):
        word_lists = train_word_lists + test_word_lists
        features = [sent2features(s) for s in word_lists]

        all_features = []
        for f in features:
            for ff in f:
                all_features.append(ff)
        v = DictVectorizer(sparse=True)
        X = v.fit_transform(all_features)

        pred_tag_lists = self.model.predict(X)

        all_words = []
        train_word_count = 0
        for t in train_word_lists:
            for tt in t:
                all_words.append(tt)
                train_word_count += 1
        for t in test_word_lists:
            for tt in t:
                all_words.append(tt)

        pred_tag_lists = pred_tag_lists[train_word_count:]
        for i in range(0, len(pred_tag_lists) - 1):
            if pred_tag_lists[i] == "O":
                if pred_tag_lists[i + 1].startswith("M-") or pred_tag_lists[i + 1].startswith("E-"):
                    pos: str = pred_tag_lists[i + 1][2:]
                    pred_tag_lists[i + 1] = "B-" + pos
            elif pred_tag_lists[i].startswith("B-") or pred_tag_lists[i].startswith("M-"):
                pos: str = pred_tag_lists[i][2:]
                if pred_tag_lists[i + 1].startswith("M-"):
                    pred_tag_lists[i + 1] = "M-" + pos
                else:
                    pred_tag_lists[i + 1] = "E-" + pos
            elif pred_tag_lists[i].startswith("E-"):
                pos: str = pred_tag_lists[i][2:]
                if pred_tag_lists[i + 1].startswith("M-"):
                    pred_tag_lists[i + 1] = "B-" + pos
                elif pred_tag_lists[i + 1].startswith("E-"):
                    pred_tag_lists[i + 1] = "O"

        return pred_tag_lists


# 6. 利用新数据重新训练和测试HMM、ME和CRF模型
请在下面的Cell中：利用给定的ner_clue_data目录下的训练数据文件train.txt分别训练HMM、ME和CRF模型,并使用dev.txt文件里的数据来测试三个模型。输出每个模型在dev.txt数据上的测试结果（每个模型对应一个结果文件），在Notebook中输出每个模型对应的每种实体以及总体的Precision、Recall和F1值。（6分）

In [16]:
# 在这里实现利用新数据重新训练和测试HMM、ME和CRF模型

# ===== HMM =====

train_word_lists, train_tag_lists, word2id, tag2id = data_build(file_name="ner_clue_data/train_char.txt", make_vocab=True)
test_word_lists, test_tag_lists = data_build(file_name="ner_clue_data/dev_char.txt", make_vocab=False)

""" HMM 参数构建 """
import numpy as np
# N: 状态数，这里对应存在的标注的种类 
# M: 观测数，这里对应有多少不同的字
N, M = len(tag2id), len(word2id)
# 状态转移概率矩阵 A[i][j]表示从i状态转移到j状态的概率
A = np.zeros(shape=(N, N), dtype=float)
# 观测概率矩阵, B[i][j]表示i状态下生成j观测的概率
B = np.zeros(shape=(N, M), dtype=float)
# 初始状态概率  Pi[i]表示初始时刻为状态i的概率
Pi = np.zeros(shape=N, dtype=float)

""" 构建转移概率矩阵 """
for tag_list in train_tag_lists:
    seq_len = len(tag_list)
    for i in range(seq_len - 1):
        current_tagid = tag2id[tag_list[i]]
        next_tagid = tag2id[tag_list[i+1]]
        A[current_tagid][next_tagid] += 1
A[A == 0.] = 1e-10  # 平滑处理
A = A / np.sum(a=A, axis=1, keepdims=True)

""" 构建观测概率矩阵 """
for tag_list, word_list in zip(train_tag_lists, train_word_lists):
    assert len(tag_list) == len(word_list)
    for tag, word in zip(tag_list, word_list):
        tag_id = tag2id[tag]
        word_id = word2id[word]
        B[tag_id][word_id] += 1
B[B == 0.] = 1e-10  # 平滑处理
B = B / np.sum(a=B, axis=1, keepdims=True)

""" 构建初始状态概率 """
for tag_list in train_tag_lists:
    init_tagid = tag2id[tag_list[0]]
    Pi[init_tagid] += 1
Pi[Pi == 0.] = 1e-10  # 平滑处理
Pi = Pi / np.sum(a=Pi)

""" 利用HMM识别ner_char_data目录下test.txt中的数据"""
pred_tag_lists = []
for word_list in test_word_lists:
    pred_tag_list = viterbi(word_list, word2id, tag2id)
    pred_tag_lists.append(pred_tag_list)

pred_tag_lists_all=[]
for tl in pred_tag_lists:
    for t in tl:
        pred_tag_lists_all.append(t)
for i in range(0, len(pred_tag_lists_all) - 1):
    if pred_tag_lists_all[i] == "O":
        if pred_tag_lists_all[i + 1].startswith("M-") or pred_tag_lists_all[i + 1].startswith("E-"):
            pos: str = pred_tag_lists_all[i + 1][2:]
            pred_tag_lists_all[i + 1] = "B-" + pos
    elif pred_tag_lists_all[i].startswith("B-") or pred_tag_lists_all[i].startswith("M-"):
        pos: str = pred_tag_lists_all[i][2:]
        if pred_tag_lists_all[i + 1].startswith("M-"):
            pred_tag_lists_all[i + 1] = "M-" + pos
        else:
            pred_tag_lists_all[i + 1] = "E-" + pos
    elif pred_tag_lists_all[i].startswith("E-"):
        pos: str = pred_tag_lists_all[i][2:]
        if pred_tag_lists_all[i + 1].startswith("M-"):
            pred_tag_lists_all[i + 1] = "B-" + pos
        elif pred_tag_lists_all[i + 1].startswith("E-"):
            pred_tag_lists_all[i + 1] = "O"

with open("result-HMM.txt", "w") as f:
    with open("ner_clue_data/dev_char.txt", "r") as tf:
        count = 0
        for line in tf:
            if line == "\n":
                f.write("\n")
            else:
                f.write("字 " + pred_tag_lists_all[count] + "\n")
                count += 1


In [18]:
# ===== ME =====

train_word_lists, train_tag_lists, train_word2id, train_tag2id = data_build(file_name="ner_clue_data/train_char.txt",
                                                                            make_vocab=True)
test_word_lists, test_tag_lists, test_word2id, test_tag2id = data_build(file_name="ner_clue_data/dev_char.txt",
                                                                        make_vocab=True)

# 训练模型
me_model = MEModel()
me_model.train(train_word_lists, train_tag_lists, test_word_lists, test_tag_lists, test_word2id)

pred_tag_lists: list = me_model.test(train_word_lists, test_word_lists)

with open("result-ME.txt", "w") as f:
    with open("ner_clue_data/dev_char.txt", "r") as tf:
        count = 0
        for line in tf:
            if line == "\n":
                f.write("\n")
            else:
                f.write("字 " + pred_tag_lists[count] + "\n")
                count += 1


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [20]:
# ===== CRF =====

train_word_lists, train_tag_lists, word2id, tag2id = data_build(file_name="ner_clue_data/train_char.txt", make_vocab=True)
test_word_lists, test_tag_lists = data_build(file_name="ner_clue_data/dev_char.txt", make_vocab=False)

crf_model = CRFModel()
crf_model.train(train_word_lists, train_tag_lists)

pred_tag_lists = crf_model.test(test_word_lists)
pred_tag_lists_all=[]
for tl in pred_tag_lists:
    for t in tl:
        pred_tag_lists_all.append(t)
for i in range(0, len(pred_tag_lists_all) - 1):
    if pred_tag_lists_all[i] == "O":
        if pred_tag_lists_all[i + 1].startswith("M-") or pred_tag_lists_all[i + 1].startswith("E-"):
            pos: str = pred_tag_lists_all[i + 1][2:]
            pred_tag_lists_all[i + 1] = "B-" + pos
    elif pred_tag_lists_all[i].startswith("B-") or pred_tag_lists_all[i].startswith("M-"):
        pos: str = pred_tag_lists_all[i][2:]
        if pred_tag_lists_all[i + 1].startswith("M-"):
            pred_tag_lists_all[i + 1] = "M-" + pos
        else:
            pred_tag_lists_all[i + 1] = "E-" + pos
    elif pred_tag_lists_all[i].startswith("E-"):
        pos: str = pred_tag_lists_all[i][2:]
        if pred_tag_lists_all[i + 1].startswith("M-"):
            pred_tag_lists_all[i + 1] = "B-" + pos
        elif pred_tag_lists_all[i + 1].startswith("E-"):
            pred_tag_lists_all[i + 1] = "O"


with open("result-CRF.txt", "w") as f:
    with open("ner_clue_data/dev_char.txt", "r") as tf:
        count = 0
        for line in tf:
            if line == "\n":
                f.write("\n")
            else:
                f.write("字 " + pred_tag_lists_all[count] + "\n")
                count += 1

In [21]:
print("HMM", end=" ")
evaluate("ner_clue_data/dev_char.txt", "result-HMM.txt")
print("ME ", end=" ")
evaluate("ner_clue_data/dev_char.txt", "result-ME.txt")
print("CRF", end=" ")
evaluate("ner_clue_data/dev_char.txt", "result-CRF.txt")

HMM presision = 0.5543619791666666, recall = 0.5243226600985221, f = 0.5389240506329114
ME  presision = 0.6031901041666666, recall = 0.48788836229594523, f = 0.5394468704512372
CRF presision = 0.6787109375, recall = 0.7710798816568047, f = 0.7219529085872576


In [22]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
# !ls /home/aistudio/data

In [23]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
# !ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
# !mkdir /home/aistudio/external-libraries
# !pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
# import sys 
# sys.path.append('//home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 